In [2]:
from recompilation_functions import *
from randomized_compiling_functions import *

In [3]:
#creation of the circuits using recompiled tensor network method

nb_qubits = 4#system size
depth=20 #depth of the recompiled circuit
t = 10#time of the simulation (quench)
mean=0 #mean value of the coupling strength's gaussian distribution
sigma=1 #standard deviation of the coupling strength's gaussian distribution
nr=100 #number of disorder realizations
omegas=[0.28,0.38,0.63,0.86] #qubits frequencies


r=1#initial disorder realization
for d in range(r,nr+1):


    #random interaction
    J=np.zeros((nb_qubits,nb_qubits),dtype=complex)
    for i in range(nb_qubits):
        for j in range(i+1,nb_qubits):
            J[i,j]= np.random.normal(loc = mean, scale = np.sqrt(0.5 * sigma/nb_qubits), size = (1, 2)).view(np.complex)[0][0]
            J[j,i]=np.conjugate(J[i,j])

    #Saving the matrix of random coupling_strength of the system for one disorder realization
    #WARNING IF YOU UNCOMMENT THE LINE BELOW, THE PREVIOUS MATRIX OF THE CORRESPONDING DISORDER REALIZATION WILL BE ERASED
    #np.save('data/disorder_realization_'+str(d)+'/random_interaction_'+str(d),J)

    np.set_printoptions(precision=5)
    print(d)
    print(J)

    #exact unitary
    unitary = linalg.expm(-1j*random_gaussian_interactions(J,omegas)*t)

    
    #initial state
    qc=QuantumCircuit(nb_qubits)
    for i in range(nb_qubits):
        qc.rx(np.pi/2,i)
    qc.barrier()
    
    #circuit recompiled
    qc_recompiled=tensor_network_recompilation(unitary, depth, ansatz_fct = ansatz_circuit, two_qubit_gate = 'CX', n_iter = 5000, n_hop = 20, temperature = 10, print_bool = True)  
    qc.append(qc_recompiled,[i for i in range(nb_qubits)])
    
    #measurement in the y-direction
    qc_meas=add_measure(qc)

    #save the quantum circuit
    #WARNING IF YOU UNCOMMENT THE LINE BELOW, THE PREVIOUS CIRCUIT OF THE CORRESPONDING DISORDER REALIZATION WILL BE ERASED
    #qc_meas.qasm(formatted=True,filename='data/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/recompiled_circuit_disorder='+str(d)+'_T='+str(round(t,1))+'.qasm')

C:\Users\hugop\AppData\Local\Temp\ipykernel_15656\145221726.py:20: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  J[i,j]= np.random.normal(loc = mean, scale = np.sqrt(0.5 * sigma/nb_qubits), size = (1, 2)).view(np.complex)[0][0]


1
[[ 0.     +0.j      -0.31179+0.19076j  0.38648+0.47359j  0.16184-0.06752j]
 [-0.31179-0.19076j  0.     +0.j      -0.01749+0.16363j -0.40915-0.20559j]
 [ 0.38648-0.47359j -0.01749-0.16363j  0.     +0.j       0.14707-0.24773j]
 [ 0.16184+0.06752j -0.40915+0.20559j  0.14707+0.24773j  0.     +0.j     ]]
Original number of CNOT : 271
Ansatz number of CNOT : 60


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
  0%|                                                                                       | 0/100000 [00:00<?, ?it/s]C:\Users\hugop\.conda\envs\thermal\lib\site-packages\jax\_src\numpy\array_methods.py:728: UserWarning: Explicitly requested dtype <class 'jax.numpy.complex128'> requested in astype is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return getattr(self.aval, name).fun(self, *args, **kwargs)
+0.005253851414 [best: +0.003187835217] :  10%|███▎                              | 9695/100000 [05:40<52:55, 28.44it/s]
C:\Users\hugop\AppData\Roaming\Python\Python39\site-packages\autoray\autoray.py:1052: UserWarning: Explicitly requested dtype <class 'jax.numpy.complex128'> requested in astype is not avail

Fidelity (tensor): 99.6985597494 %
Fidelity (circuit): 99.7013940323 %


In [4]:
#creation of the RC circuits for each disorder realization

%%capture
nr=1#number of disorder realizations
r=1
t=10 #time of the simulation 
nb_twirls=100#number of RC circuits per realization
coupling_map=[[0,1],[1,0],[1,2],[2,1],[2,3],[3,2]]#linear coupling map between the qubits
for d in range(r,nr+1):
    qc=QuantumCircuit.from_qasm_file('data/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/recompiled_circuit_disorder_'+str(d)+'_T='+str(round(t,1))+'.qasm')
    for k in range(nb_twirls):
        qc2=crosstalk_twirl(qc,coupling_map)
        #save circuits, if you uncomment the line below the previous circuits already saved will be erased
        #qc2.qasm(formatted=True,filename='data/disorder_realization_'+str(d)+'/T='+str(round(t,1))+'/RC/recompiled_circuit_disorder_'+str(d)+'_T='+str(round(t,1))+'_RC_'+str(k)+'.qasm')